<div style="text-align: left;"><img src="https://www.juliabox.org/assets/img/juliacloudlogo.png" style="margin: 0px 0px 0px 0px; padding-right: 20px;width: 80px; float: left;" title="" alt="" /></div>
<img src="http://dmkpress.com/images/cms/thumbs/a5b0aeaa3fa7d6e58d75710c18673bd7ec6d5f6d/978-5-97060-370-3_270_369__100.jpg" style="margin: 0px 0px 5px 20px; width: 100px; float: right;" title="" alt="" />
Всестороннее введение в новый язык программирования для научно-технических вычислений [Julia](http://julialang.org/) в книге Малколма Шеррингтона, Packt Publishing, июль 2015.

<h1>Осваиваем язык Julia</h1><br />

Совершенствование мастерства в области аналитики и программирования при помощи Julia в целях решения задач комплексной обработки данных
<div style="text-align: left;font-size:8pt;padding-top:10px;">Программный код Julia (v0.4.5) протестирован в Windows 8.1/10 и Linux/Lubuntu 16.4</div>
<div style="text-align: left;"><h1>Глава 8. Базы данных</h1></div>

## CouchDB

In [1]:
using Requests, JSON

url = "http://localhost:5984"
uu = Requests.get(url);
json = JSON.parse(bytestring(uu.data))

Dict{AbstractString,Any} with 4 entries:
  "uuid"    => "07fb3f7ef7db5835f07828f2c233474b"
  "couchdb" => "Welcome"
  "version" => "1.6.1"
  "vendor"  => Dict{AbstractString,Any}("name"=>"The Apache Software Foundation…

In [16]:
# Создать новую базу данных

# run(`curl -X PUT http://localhost:5984/quotes_ru`)  

url = "http://localhost:5984/quotes"
Requests.put(url); 

In [12]:
#run(`curl http://localhost:5984/_all_dbs`)

url = "http://localhost:5984/_all_dbs"
uu = Requests.get(url);
json = JSON.parse(bytestring(uu.data))   

3-element Array{Any,1}:
 "_replicator"
 "_users"     
 "quotes"     

In [3]:
# curl -H 'Content-Type: application/json' -X POST http://127.0.0.1:5984/quotes -d '{category:"...",author:"...", "quote":"..."}'

url = "http://localhost:5984/quotes"
json = Dict("category" => "Computing", 
            "author" => "Scott's Law", 
            "quote" => "Adding manpower to a late software project makes it later") 
uu = Requests.post(url, json = json)

Response(201 Created, 11 headers, 95 bytes in body)

In [13]:
cd(joinpath(homedir(),"julia_projects","data"))
include("quotes_arr.jl")
 
using Requests, JSON

cdb = "http://localhost:5984";
qc = cdb * "/quotes";

for json in quotes_arr
  Requests.post("$qc", json=json)
end

println("Данные успешно внесены.")

Данные успешно внесены.


In [4]:
cdb = "http://localhost:5984"
qc_all = cdb * "/quotes/_all_docs"

res = bytestring(Requests.get(qc_all).data);
json = JSON.parse(res) 

Dict{AbstractString,Any} with 3 entries:
  "rows"       => Any[Dict{AbstractString,Any}("key"=>"a1b231f6d4193e03702d181b…
  "offset"     => 0
  "total_rows" => 2

In [5]:
json["total_rows"]   

2

In [6]:
json["rows"][2]      # Выделить конкретную запись

Dict{AbstractString,Any} with 3 entries:
  "key"   => "a1b231f6d4193e03702d181be90058d9"
  "id"    => "a1b231f6d4193e03702d181be90058d9"
  "value" => Dict{AbstractString,Any}("rev"=>"1-c39a3f858486377c80483304e012ce4…

In [7]:
qc = cdb * "/quotes";

key = json["rows"][2]["key"];
rev = json["rows"][2]["value"]["rev"];

JSON.parse(bytestring(Requests.get("$qc/$key").data))

Dict{AbstractString,Any} with 5 entries:
  "quote"    => "I can resist everything but temptation"
  "_rev"     => "1-c39a3f858486377c80483304e012ce47"
  "author"   => "Oscar Wilde"
  "_id"      => "a1b231f6d4193e03702d181be90058d9"
  "category" => "Books & Plays"

In [ ]:
Requests.delete("$qc/$key?rev=$rev")

In [ ]:
json = Dict("category" => "Books & Plays", "author" => "Oscar Wilde", "quote" => "I can resist everything but temptation")

Requests.delete("$qc/$key?rev=$rev")
Requests.post("$qc", json=json)